In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_columns', None)  # Display all columns

In [2]:
#!pip install category_encoders

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc

root = 'C:/Users/Pratham/Documents/GitHub/instacart-customer-purchase-prediction/data/'

In [4]:
order_products = pd.read_parquet(root + 'order_products.parquet')

In [5]:
# filter data where eval_set = 'prior'

order_products = order_products[order_products.eval_set == 'prior']


In [ ]:
# 1. Total orders by a user
user_total_orders = order_products.groupby('user_id')['order_number'].max().reset_index()
user_total_orders.rename(columns={'order_number': 'total_orders_by_user'}, inplace=True)

# 2. User’s overall reorder percentage
user_reorder_percentage = order_products.groupby('user_id')['reordered'].mean().reset_index()
user_reorder_percentage.rename(columns={'reordered': 'user_reorder_percentage'}, inplace=True)

# 3. User's average days-since-prior-order
user_avg_days_since_prior = order_products.groupby('user_id')['days_since_prior_order'].mean().reset_index()
user_avg_days_since_prior.rename(columns={'days_since_prior_order': 'user_avg_days_since_prior_order'}, inplace=True)

# 4. Total times the product was reordered
product_reorder_count = order_products.groupby('product_id')['reordered'].sum().reset_index()
product_reorder_count.rename(columns={'reordered': 'total_times_product_reordered'}, inplace=True)

# 5. Product reorder percentage
product_reorder_percentage = order_products.groupby('product_id')['reordered'].mean().reset_index()
product_reorder_percentage.rename(columns={'reordered': 'product_reorder_percentage'}, inplace=True)


# 6. User’s most frequent order day-of-week
user_most_frequent_order_dow = order_products.groupby('user_id')['order_dow'].agg(lambda x: x.mode()[0]).reset_index()
user_most_frequent_order_dow.rename(columns={'order_dow': 'user_most_frequent_order_dow'}, inplace=True)

# 7. User’s most frequent order hour-of-day
user_most_frequent_order_hour = order_products.groupby('user_id')['order_hour_of_day'].agg(lambda x: x.mode()[0]).reset_index()
user_most_frequent_order_hour.rename(columns={'order_hour_of_day': 'user_most_frequent_order_hour'}, inplace=True)

# Merge all features into a single dataframe
features = user_total_orders
features = features.merge(user_reorder_percentage, on='user_id', how='left')
features = features.merge(user_avg_days_since_prior, on='user_id', how='left')
features = features.merge(user_most_frequent_order_dow, on='user_id', how='left')
features = features.merge(user_most_frequent_order_hour, on='user_id', how='left')

product_features = product_reorder_count.merge(product_reorder_percentage, on='product_id', how='left')


In [7]:
features.head()

,user_id,total_orders_by_user,user_reorder_percentage,user_avg_days_since_prior_order,user_most_frequent_order_dow,user_most_frequent_order_hour
0,1,10,0.694915,20.259260,4,7
1,2,14,0.476923,15.967033,2,9
2,3,12,0.625000,11.487180,0,16
3,4,5,0.055556,15.357142,4,15
4,5,4,0.378378,14.500000,3,18


In [8]:
product_features.head()

,product_id,total_times_product_reordered,product_reorder_percentage
0,1,1136,0.613391
1,2,12,0.133333
2,3,203,0.732852
3,4,147,0.446809
4,5,9,0.600000


In [9]:
order_products_train = pd.read_parquet(root + 'order_products.parquet')
order_products_train = order_products_train[order_products_train.eval_set == 'train']

In [10]:
order_products_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1384617 entries, 32434489 to 33819105
Data columns (total 15 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   order_id                1384617 non-null  int32  
 1   product_id              1384617 non-null  int32  
 2   add_to_cart_order       1384617 non-null  int16  
 3   reordered               1384617 non-null  int8   
 4   product_name            1384617 non-null  object 
 5   aisle_id                1384617 non-null  int16  
 6   department_id           1384617 non-null  int8   
 7   aisle                   1384617 non-null  object 
 8   department              1384617 non-null  object 
 9   user_id                 1384617 non-null  int32  
 10  eval_set                1384617 non-null  object 
 11  order_number            1384617 non-null  int8   
 12  order_dow               1384617 non-null  int8   
 13  order_hour_of_day       1384617 non-null  int8   
 14 

In [11]:
order_products_train=order_products_train.merge(features, on='user_id', how='left').merge(product_features, on='product_id', how='left')

In [12]:
order_products_train.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,total_orders_by_user,user_reorder_percentage,user_avg_days_since_prior_order,user_most_frequent_order_dow,user_most_frequent_order_hour,total_times_product_reordered,product_reorder_percentage
0,1,49302,1,1,Bulgarian Yogurt,120,16,yogurt,dairy eggs,112108,train,4,4,10,9.0,3,0.428571,10.076923,1,10,101.0,0.619632
1,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,other creams cheeses,dairy eggs,112108,train,4,4,10,9.0,3,0.428571,10.076923,1,10,3192.0,0.713775
2,1,10246,3,0,Organic Celery Hearts,83,4,fresh vegetables,produce,112108,train,4,4,10,9.0,3,0.428571,10.076923,1,10,12498.0,0.524553
3,1,49683,4,0,Cucumber Kirby,83,4,fresh vegetables,produce,112108,train,4,4,10,9.0,3,0.428571,10.076923,1,10,67313.0,0.691702
4,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15,canned meat seafood,canned goods,112108,train,4,4,10,9.0,3,0.428571,10.076923,1,10,312.0,0.477795


In [13]:
order_products_train.isna().sum()

order_id                           0
product_id                         0
add_to_cart_order                  0
reordered                          0
product_name                       0
aisle_id                           0
department_id                      0
aisle                              0
department                         0
user_id                            0
eval_set                           0
order_number                       0
order_dow                          0
order_hour_of_day                  0
days_since_prior_order             0
total_orders_by_user               0
user_reorder_percentage            0
user_avg_days_since_prior_order    0
user_most_frequent_order_dow       0
user_most_frequent_order_hour      0
total_times_product_reordered      9
product_reorder_percentage         9
dtype: int64

In [14]:
order_products_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1384617 entries, 0 to 1384616
Data columns (total 22 columns):
 #   Column                           Non-Null Count    Dtype  
---  ------                           --------------    -----  
 0   order_id                         1384617 non-null  int32  
 1   product_id                       1384617 non-null  int32  
 2   add_to_cart_order                1384617 non-null  int16  
 3   reordered                        1384617 non-null  int8   
 4   product_name                     1384617 non-null  object 
 5   aisle_id                         1384617 non-null  int16  
 6   department_id                    1384617 non-null  int8   
 7   aisle                            1384617 non-null  object 
 8   department                       1384617 non-null  object 
 9   user_id                          1384617 non-null  int32  
 10  eval_set                         1384617 non-null  object 
 11  order_number                     1384617 non-null 

In [15]:
# Impute missing values (appropriate strategy)
order_products_train.fillna({'user_avg_days_since_prior_order': features['user_avg_days_since_prior_order'].median(),
                       'user_reorder_percentage': 0,
                       'total_times_product_reordered': 0,
                       'product_reorder_percentage': 0,
                       }, inplace=True)


In [16]:
order_products_train.isna().sum()

order_id                           0
product_id                         0
add_to_cart_order                  0
reordered                          0
product_name                       0
aisle_id                           0
department_id                      0
aisle                              0
department                         0
user_id                            0
eval_set                           0
order_number                       0
order_dow                          0
order_hour_of_day                  0
days_since_prior_order             0
total_orders_by_user               0
user_reorder_percentage            0
user_avg_days_since_prior_order    0
user_most_frequent_order_dow       0
user_most_frequent_order_hour      0
total_times_product_reordered      0
product_reorder_percentage         0
dtype: int64

In [17]:
order_products_train.to_parquet(root + 'feature_engineer_dataset.parquet')